# [CSCI 3397/PSYC 3317] Lab 3a: Image Preprocessing

**Posted:** Tuesday, February 1, 2022

**Due:** Thursday, February 10, 2022

__Total Points__: 2 pts

__Submission__: please rename the .ipynb file as __\<your_username\>_lab3a.ipynb__ before you submit it to canvas. Example: weidf_lab3a.ipynb.

# Microscopy image applications with tools in Lec. 3

- Transfer function
    - 1. Image adjustment

- Filtering
    - 2. Background subtraction
    - 3. Illumination correction
    - 4. Blob filter (Laplacian of Gaussian)

# 1. Fluorescent image visualization

In [ ]:
import matplotlib.pyplot as plt
from imageio import volread
import numpy as np

# (1) Naive approach: Recap Lab1b

def volToRGB(vol):
    im_size = vol.shape
    # load channels into different RGB channel
    im = np.zeros([im_size[1], im_size[2], 3], np.uint8)
    for cid in range(3):    
        im[:, :, cid] = vol[cid]
        print('channel %d value: %d-%d' % (cid, vol[cid].min(), vol[cid].max()))
    return im


# unlike imread, volread loads image volume
vol = volread('lab3/CovidRollingball.tif')
print('vol size: ', vol.shape)
print('The first dimension is the channel dimension')

# display image
plt.imshow(volToRGB(vol))
plt.axis('off')
plt.title('First 3 channels (naive)');

print('--------------------------')
print('It looks horrible...')
print('Error I: the channel type is uint16 with values bigger than 255')
print('Error II: need a transfer function to adjust the range of the image')

In [ ]:
# (2) recap the transfer function in Lec. 3
# from now on, we will use imAdjust for visualization: imshow(imAdjust(I)) instead of imshow(I)


def imAdjust(I, thres=[1,99]):
    # compute percentile: remove too big or too small values
    I_low, I_high = np.percentile(I.reshape(-1), thres)
    # thresholding
    I[I > I_high] = I_high
    I[I < I_low] = I_low
    # scale to 0-1
    I = (I.astype(float)- I_low)/ (I_high-I_low)
    # convert it to uint8
    I = (I * 255).astype(np.uint8)
    return I

def volAdjust(vol, thres=[1,99]):
    # apply imAdjust to each channel
    vol_adjust = vol.copy()
    for cid in range(vol.shape[0]):
        vol_adjust[cid] = imadjust(vol[cid])
    return vol_adjust


# display image
plt.imshow(volToRGB(volAdjust(vol)))
plt.axis('off')
plt.title('First 3 channels (imadjust)');

# 2. Background subtraction

We will learn more about morphological filtering and top-hat filter in Lec. 6. 
Here, only need to 
- observe that the top-hat operation makes the object border smoother [[link]](https://micro.magnet.fsu.edu/primer/java/digitalimaging/russ/tophatfilter/index.html)
- have fun with the interactive slide bar, which is hugely useful when you want to tweak parameters! [[link]](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from imageio import volread
import cv2

# let's make a widget:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


# take a single channel and focus on one cell
# global variable
I = volread('lab3/CovidRollingball.tif')[1, 50:170,150:250]


def tophatFilter(filterSize):
    # top-hat filter
    kernel = np.ones([filterSize, filterSize])
    tophat_img = cv2.morphologyEx(I, cv2.MORPH_TOPHAT, kernel)
    
    plt.figure(figsize=(8, 16))
    plt.subplot(131)
    plt.imshow(imAdjust(I), cmap='gray')
    plt.title('input image')
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(imAdjust(tophat_img), cmap='gray')
    plt.title('tophat: border artifacts')
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(imAdjust(I-tophat_img), cmap='gray')
    plt.title('input - tophat')
    plt.axis('off')

    plt.show()
    
interact(tophatFilter, filterSize=widgets.IntSlider(min=3, max=31, step=1, value=5));


# 3. Illumination correction
Load the pre-computed background pattern. This is a fixed microscope artifact and we can remove it computationally

In [ ]:
from imageio import imread,imsave
FFC = imread('lab3/FFC.tif');
sz = FFC.shape
# for storage
print('tile size:', FFC.shape)
print('value range:', FFC.min(), FFC.max())

plt.imshow(FFC,cmap='gray');plt.axis('off')
plt.show()

In [ ]:
I_stitched = np.zeros([sz[0]*7, sz[1]*13], np.uint16);

for row in range(7):       
    for col in range(13):
        # well, different ppl have different preference for the starting index: 0 or 1
        I = imread('lab3/unstitchedTiles/FITC_x_%d_y_%d.tif'% (1+col, 1+row));
        I_stitched[row*sz[0]: (row+1)*sz[0], col*sz[1]: (col+1)*sz[1]] = I

plt.figure(figsize=(16, 32))
plt.imshow(imAdjust(I_stitched), cmap='gray')
plt.show()

In [ ]:
I_stitched_bg = np.zeros([sz[0]*7, sz[1]*13], np.uint16);

for row in range(7):       
    for col in range(13):
        #### divide the lighting pattern
        I = imread('lab3/unstitchedTiles/FITC_x_%d_y_%d.tif'% (1+col, 1+row)).astype(float)/FFC;
        I_stitched_bg[row*sz[0]: (row+1)*sz[0], col*sz[1]: (col+1)*sz[1]] = I

plt.figure(figsize=(16, 32))
plt.imshow(imAdjust(I_stitched_bg), cmap='gray')
plt.show()

In [ ]:
# for a zoom-in comparison
# There are vertical dark stripes for the naive stitching

plt.figure(figsize=(16, 32))
plt.subplot(121)
plt.imshow(imAdjust(I_stitched[:, :1500]), cmap='gray')
plt.axis('off');plt.title('Naive stitching')
plt.subplot(122)
plt.imshow(imAdjust(I_stitched_bg[:, :1500]), cmap='gray')
plt.axis('off');plt.title('Stitching with bg subtraction')
plt.show()

# 4. Blob detection with LoG filter
We learned about Gaussian filter (G) for denoising, difference of Gaussian filter (DoG) for robust edge detection.
Today, let's look at Laplacian of Gaussian filter (LoG) for blob detection. 

[[More maths explanation]](https://en.wikipedia.org/wiki/Blob_detection)

Maths formula: <img src="https://cdn-images-1.medium.com/max/800/0*5uzMO2ZULkv6Pcpr.gif">

Kernel visualization: <img src="https://cdn-images-1.medium.com/max/800/1*I9LJae_DIAZe6q4AkdayyA.png">

In [ ]:
from imageio import volread
from skimage.feature import blob_log
import cv2

def LoG(sigma):
    #window size 
    n = np.ceil(sigma*6)
    y,x = np.ogrid[-n//2:n//2+1,-n//2:n//2+1]
    y_filter = np.exp(-(y*y/(2.*sigma*sigma)))
    x_filter = np.exp(-(x*x/(2.*sigma*sigma)))
    final_filter = (-(2*sigma**2) + (x*x + y*y) ) *  (x_filter*y_filter) * (1/(2*np.pi*sigma**4))
    return final_filter

I = volread('lab3/cycif.tif')[1]
I_LoG = cv2.filter2D(I, -1, LoG(1.6))


plt.figure(figsize=(16, 32))
plt.subplot(121)
plt.imshow(imAdjust(I), cmap='gray')
plt.axis('off');plt.title('input image')
plt.subplot(122)
plt.imshow(imAdjust(I_LoG), cmap='gray')
plt.axis('off');plt.title('LoG result: bright blobs left')
plt.show()



# [2 pts] Exercise
As pset1 is coming up, we'll make this lab light-weight :)

## (1) [2 pts] Make your own slider for parameter selection!

Let's find the pixel value threshold $\theta$ so the brightfield image of bacteria: any pixel value bigger than $\theta$ is assign to 1 and 0 otherwise.

In [ ]:
from imageio import imread

# example: with a bad threshold, you can't get all bacteria
I = imread('lab3/brightfield_bacteria.jpg')

th = 70

plt.figure(figsize=(16, 32))
plt.subplot(121)
plt.imshow(I, cmap='gray')
plt.axis('off');plt.title('input image')
plt.subplot(122)
plt.imshow(I>th, cmap='gray')
plt.axis('off');plt.title('bad thresholded binary image')
plt.show()

In [ ]:
# make a slider so that ppl can manually pick the value needed to get all bacteria
### Your code starts
### Your code ends